In [50]:
import Random
import Dates
##KNOWN ISSUES
##PERIODIC BOUNDARY - CRASH WHILE CHANGING LANE TO THE RIGHT WHILE PERSON FROM BEGINNING OF AN ARRAY CHANGES LANE ALSO
##TEMPORARY FIX - NO CHANGING LANES FROM INDEX 1-5
##CRASHING CARS WHILE SOMEBODY CHANGES LANE WITH VELOCITY 0-1. TEMPORARY SOLUTION - ONLY CHANGE LANES WHILE YOUR SPEED > 1

In [55]:
function mcs_closed(vehicles, velocities, lane_changes, max_velocity, min_velocity, p, politeness_factor)
    """
    Performs 1 MCS. Velocity updates happen in parallel for every car.
    """
    
    update_velocities_closed(vehicles, velocities, lane_changes, max_velocity, min_velocity, p, politeness_factor)
    
    print_state(vehicles, velocities, lane_changes)
    
    update_positions_closed(vehicles, velocities, lane_changes)
    
end

function mcs_closed_save(vehicles, velocities, lane_changes, max_velocity, min_velocity, p, politeness_factor, io)
    """
    Performs 1 MCS. Velocity updates happen in parallel for every car.
    """
    
    update_velocities_closed(vehicles, velocities, lane_changes, max_velocity, min_velocity, p, politeness_factor)
    
    print_state_save(vehicles, velocities, lane_changes, io)
    
    update_positions_closed(vehicles, velocities, lane_changes)
    
end

mcs_closed_save (generic function with 1 method)

In [41]:
function mcs_open(vehicles, velocities, lane_changes, max_velocity, min_velocity, p, politeness_factor)
    """
    Performs 1 MCS. Velocity updates happen in parallel for every car.
    """
    
    update_velocities_open(vehicles, velocities, lane_changes, max_velocity, min_velocity, p, politeness_factor)
    
    print_state(vehicles, velocities, lane_changes)
    
    update_positions_open(vehicles, velocities, lane_changes)
    
end

function mcs_open_save(vehicles, velocities, lane_changes, max_velocity, min_velocity, p, politeness_factor, io)
    """
    Performs 1 MCS. Velocity updates happen in parallel for every car.
    """
    
    update_velocities_open(vehicles, velocities, lane_changes, max_velocity, min_velocity, p, politeness_factor)
    
    print_state_save(vehicles, velocities, lane_changes, io)
    
    update_positions_open(vehicles, velocities, lane_changes)
    
end

mcs_open_save (generic function with 1 method)

In [57]:
function update_velocities_closed(vehicles, velocities, lane_changes, max_velocity, min_velocity, p, politeness_factor)
    """
    Agent behaviour checklist 
    ## SAFELY CHANGE LANE TO THE RIGHT
    ## ACCELERATE
    ## SAFELY TRY TO PASS
    ## TRY A-HOLE PASS LEFT (NOT DONE YET)
    ## TRY A-HOLE PASS RIGHT (NOT DONE YET)
    If no move was chosen, agent has to slow down to nearest person in front.
  
    """
    
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    next_vehicle=fill(0, (lines, 1))
    
    
    for i = 1:length
        for j = 1:lines
            if next_vehicle[j]==0 && vehicles[j,i]!=0
                next_vehicle[j] = i + length
            end
        end
    end
    for j = 1:lines
        if next_vehicle[j]==0
            next_vehicle[j]=length+10 ##NIE MA NIKOGO W STANIE POCZATKOWYM
        end
    end
        
    for i = length:-1:1
        for j = lines:-1:1
            if vehicles[j, i]!=0
                move="none"
                ### SAFE LANE CHANGE - RIGHT
                if j>1 && velocities[j,i]>1 && i>4
                    situation=check_right_mirror(vehicles, velocities, next_vehicle, j, i)
                    if situation=="safe"
                        move="change_lane_right"
                    end
                end
                
                ### ACCELERATION
                if move=="none"
                    if next_vehicle[j]>i+velocities[j,i]
                        move="accelerate"
                    end
                end
                
                ### SAFE LANE CHANGE - LEFT
                if move=="none" && j<lines && i>4 && velocities[j,i]>1
                    if next_vehicle[j]<=i+velocities[j,i]
                        ## WYPRZEDZANIE
                        if next_vehicle[j+1]>i+velocities[j,i]
                            situation=check_left_mirror(vehicles, velocities, next_vehicle, j, i)
                            if situation=="safe"
                                move="change_lane_left"
                            end
                        end
                    end
                end
                
                if move=="none" && j<lines && i>4 && velocities[j,i]>1
                    if next_vehicle[j]<=i+velocities[j,i]
                        ##A-HOLE PASSING LEFT
                        if next_vehicle[j+1]>i+velocities[j,i] && vehicles[j+1, i]==0
                            if Random.rand() < 1-politeness_factor
                                move="ahole_change_lane_left"
                            end
                        end
                    end
                end
                
                if move=="none" && j>1 && i>4 && velocities[j,i]>1
                    if next_vehicle[j]<=i+velocities[j,i]
                        ##A-HOLE PASSING LEFT
                        if next_vehicle[j-1]>i+velocities[j,i] && vehicles[j-1, i]==0
                            if Random.rand() < 1-politeness_factor
                                move="ahole_change_lane_right"
                            end
                        end
                    end
                end
                
                ### SLOWING DOWN
                if move=="none"
                    if next_vehicle[j]<=i+velocities[j,i]
                        move="slow_down"
                    end
                end
                        
                      
                if move == "accelerate"
                   accelerate(velocities, i, j, next_vehicle[j], max_velocity)
                   slow_down_random(velocities, i, j, p)
                end
                
                if move == "change_lane_right"
                    lane_changes[j,i]=-1
                    slow_down_random(velocities, i, j, p)
                    next_vehicle[j-1]=i+floor(velocities[j,i]/2)
                end
                
                if move == "change_lane_left"
                    lane_changes[j,i]=1
                    slow_down_random(velocities, i, j, p)
                    next_vehicle[j+1]=i+floor(velocities[j,i]/2)
                end
                
                if move == "ahole_change_lane_left"
                    lane_changes[j,i]=1
                    slow_down_random(velocities, i, j, p)
                    next_vehicle[j+1]=i+floor(velocities[j,i]/2)
#                     println(j, i, next_vehicle)
                end
                
                if move == "ahole_change_lane_right"
                    lane_changes[j,i]=-1 
                    slow_down_random(velocities, i, j, p)
                    next_vehicle[j-1]=i+floor(velocities[j,i]/2)
#                     println(j, i, next_vehicle)
                end
                        
                if move == "slow_down"
                   slow_down_car(velocities, i, j, next_vehicle[j]) 
                   slow_down_random(velocities, i, j, p)
                end
                        
                next_vehicle[j] = i 
                
#                 if j>1 && next_vehicle[j-1]>i+velocities[j,i]+1 && vehicles[j-1, i]==0 && i>4 && velocities[j,i]>1
#                     lane_changes[j,i]=-1
#                     if next_vehicle[j-1] > i + velocities[j, i]
#                         accelerate(velocities, i, j, next_vehicle[j-1], max_velocity)
#                     else
#                         slow_down_car(velocities, i, j, next_vehicle[j-1])  
#                     end
#                     slow_down_random(velocities, i, j, p)
#                     next_vehicle[j-1]=i + ceil(velocities[j,i]/2)    
#                 else
#                     if next_vehicle[j] > i + velocities[j, i]
#                     ##HAS PLACE - ACCELERATING
#                         accelerate(velocities, i, j, next_vehicle[j], max_velocity)
#                         slow_down_random(velocities, i, j, p)  
#                     else
#                         #### WYPRZEDZANIE
#                         if j<lines && next_vehicle[j]<next_vehicle[j+1] && i>4
#                             ### ADDITIONAL CONDITION: Nastepny pojazd jedzie wolniej od naszego
#                             if next_vehicle[j]>length
#                                 pos_next_vehicle=next_vehicle[j]%length
#                             else
#                                 pos_next_vehicle=next_vehicle[j]
#                             end
#                             if velocities[j, pos_next_vehicle]<velocities[j,i]
#                                 lane_changes[j,i]=1
#                                 if next_vehicle[j+1] > i + velocities[j, i]
#                                     accelerate(velocities, i, j, next_vehicle[j+1], max_velocity)
#                                     slow_down_random(velocities, i, j, p)
#                                     next_vehicle[j+1] = i + ceil(velocities[j,i]/2)
#                                 else
#                                     slow_down_car(velocities, i, j, next_vehicle[j+1])
#                                     slow_down_random(velocities, i, j, p)
#                                     next_vehicle[j+1] = i + ceil(velocities[j,i]/2)    
#                                 end
#                             end
#                         else 
#                             slow_down_car(velocities, i, j, next_vehicle[j])
#                             slow_down_random(velocities, i, j, p)      
#                         end
#                     end
#                 end                
            end
        end
    end
end

function update_positions_closed(vehicles, velocities, lane_changes)
        """
        
        """
    length = size(velocities)[2]
    lines = size(velocities)[1]
    
    # If rightmost vehicle goes through periodic boundary, avoid updating it second time by marking new position as "already checked"
    # At the beginning avoid_pos holds zeroes.
    avoid_pos=fill(0, (lines, length))
    
    # To avoid crashes occuring from passing the periodic boundary, we add to our diagram vehicles from front at the last possible moment,
    temp_update_vel=fill(0, (lines, 5))
    temp_update_veh=fill(0, (lines, 5))
    old_state_veh=copy(vehicles)
    old_state_vel=copy(velocities)
    for i = length:-1:1
        for j = lines:-1:1
            if (vehicles[j, i] != 0) && (velocities[j, i] != 0) && (avoid_pos[j,i]==0)
                if (i+velocities[j, i]) > length
                    new_pos=(i+velocities[j, i])%length
                    lane_change=lane_changes[j, i]
                    #if vehicles[j+lane_change, new_pos] == 1
                        #println("Car from $j, $i crashed into $(j+lane_change), $(i+velocities[j, i]), $lane_change, $(velocities[j,i])")
                        #println("CRASH REPORT")
                        #print_state(vehicles, velocities, lane_changes)
                    #end
                    temp_update_vel[j+lane_change, new_pos] = velocities[j, i]
                    temp_update_veh[j+lane_change, new_pos] = 1
                    ##vehicles[j+lane_change, new_pos] = 1
                    ##velocities[j+lane_change, new_pos] = velocities[j, i]
                    ##avoid_pos[j+lane_change, new_pos]=1
                    vehicles[j, i] = 0
                    velocities[j, i] = 0
                    lane_changes[j, i] = 0
                else
                    lane_change=lane_changes[j, i]
                    if vehicles[j+lane_change, (i+velocities[j, i])] == 1
                        println("Car from $j, $i crashed into $(j+lane_change), $(i+velocities[j, i]), $lane_change, $(velocities[j,i])")
                        println("CRASH REPORT")
                        print_state(vehicles, velocities, lane_changes)
                    end
                    vehicles[j+lane_change, (i+velocities[j, i])] = 1
                    velocities[j+lane_change, (i+velocities[j, i])] = velocities[j, i]
                    vehicles[j, i]=0
                    velocities[j, i]=0
                    lane_changes[j, i] = 0
                end
            end
        end
    end
    for i = 5:-1:1
        for j = lines:-1:1
            if temp_update_veh[j,i]!=0
                if vehicles[j, i] == 1
                    print_state(old_state_veh, old_state_vel, lane_changes)
                    println("Cars crashed on $j, $i")
                    println("CRASH REPORT")
                    print_state(vehicles, velocities, lane_changes)
                    print_state(temp_update_veh, temp_update_vel, lane_changes)
                end
                vehicles[j,i]=temp_update_veh[j,i]
                velocities[j,i]=temp_update_vel[j,i]
                temp_update_vel[j,i]=0
                temp_update_veh[j,i]=0
            end
        end
    end
end

update_positions_closed (generic function with 1 method)

In [31]:
function update_velocities_open(vehicles, velocities, lane_changes, max_velocity, min_velocity, p, politeness_factor)
    """
    Agent behaviour checklist 
    ## SAFELY CHANGE LANE TO THE RIGHT
    ## ACCELERATE
    ## SAFELY TRY TO PASS
    ## TRY A-HOLE PASS LEFT (NOT DONE YET)
    ## TRY A-HOLE PASS RIGHT (NOT DONE YET)
    If no move was chosen, agent has to slow down to nearest person in front.
  
    """
    
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    next_vehicle=fill(length+10, (lines, 1))
    
          
    for i = length:-1:1
        for j = lines:-1:1
            if vehicles[j, i]!=0
                move="none"
                
                if velocities[j,i]+i>length
                    move="accelerate"
                end
                ### SAFE LANE CHANGE - RIGHT
                if move=="none" && j>1 && velocities[j,i]>1 && i>4
                    situation=check_right_mirror(vehicles, velocities, next_vehicle, j, i)
                    if situation=="safe"
                        move="change_lane_right"
                    end
                end
                
                ### ACCELERATION
                if move=="none"
                    if next_vehicle[j]>i+velocities[j,i]
                        move="accelerate"
                    end
                end
                
                ### SAFE LANE CHANGE - LEFT
                if move=="none" && j<lines && i>4 && velocities[j,i]>1
                    if next_vehicle[j]<=i+velocities[j,i]
                        ## WYPRZEDZANIE
                        if next_vehicle[j+1]>i+velocities[j,i]
                            situation=check_left_mirror(vehicles, velocities, next_vehicle, j, i)
                            if situation=="safe"
                                move="change_lane_left"
                            end
                        end
                    end
                end
                
                if move=="none" && j<lines && i>4 && velocities[j,i]>1
                    if next_vehicle[j]<=i+velocities[j,i]
                        ##A-HOLE PASSING LEFT
                        if next_vehicle[j+1]>i+velocities[j,i] && vehicles[j+1, i]==0
                            if Random.rand() < 1-politeness_factor
                                move="ahole_change_lane_left"
                            end
                        end
                    end
                end
                
                if move=="none" && j>1 && i>4 && velocities[j,i]>1
                    if next_vehicle[j]<=i+velocities[j,i]
                        ##A-HOLE PASSING LEFT
                        if next_vehicle[j-1]>i+velocities[j,i] && vehicles[j-1, i]==0
                            if Random.rand() < 1-politeness_factor
                                move="ahole_change_lane_right"
                            end
                        end
                    end
                end
                
                ### SLOWING DOWN
                if move=="none"
                    if next_vehicle[j]<=i+velocities[j,i]
                        move="slow_down"
                    end
                end
                        
                      
                if move == "accelerate"
                   accelerate(velocities, i, j, next_vehicle[j], max_velocity)
                   slow_down_random(velocities, i, j, p)
                end
                
                if move == "change_lane_right"
                    lane_changes[j,i]=-1
                    slow_down_random(velocities, i, j, p)
                    next_vehicle[j-1]=i+floor(velocities[j,i]/2)
                end
                
                if move == "change_lane_left"
                    lane_changes[j,i]=1
                    slow_down_random(velocities, i, j, p)
                    next_vehicle[j+1]=i+floor(velocities[j,i]/2)
                end
                
                if move == "ahole_change_lane_left"
                    lane_changes[j,i]=1
                    slow_down_random(velocities, i, j, p)
                    next_vehicle[j+1]=i+floor(velocities[j,i]/2)
#                     println(j, i, next_vehicle)
                end
                
                if move == "ahole_change_lane_right"
                    lane_changes[j,i]=-1 
                    slow_down_random(velocities, i, j, p)
                    next_vehicle[j-1]=i+floor(velocities[j,i]/2)
#                     println(j, i, next_vehicle)
                end
                        
                if move == "slow_down"
                   slow_down_car(velocities, i, j, next_vehicle[j]) 
                   slow_down_random(velocities, i, j, p)
                end
                        
                next_vehicle[j] = i 
                
#                 if j>1 && next_vehicle[j-1]>i+velocities[j,i]+1 && vehicles[j-1, i]==0 && i>4 && velocities[j,i]>1
#                     lane_changes[j,i]=-1
#                     if next_vehicle[j-1] > i + velocities[j, i]
#                         accelerate(velocities, i, j, next_vehicle[j-1], max_velocity)
#                     else
#                         slow_down_car(velocities, i, j, next_vehicle[j-1])  
#                     end
#                     slow_down_random(velocities, i, j, p)
#                     next_vehicle[j-1]=i + ceil(velocities[j,i]/2)    
#                 else
#                     if next_vehicle[j] > i + velocities[j, i]
#                     ##HAS PLACE - ACCELERATING
#                         accelerate(velocities, i, j, next_vehicle[j], max_velocity)
#                         slow_down_random(velocities, i, j, p)  
#                     else
#                         #### WYPRZEDZANIE
#                         if j<lines && next_vehicle[j]<next_vehicle[j+1] && i>4
#                             ### ADDITIONAL CONDITION: Nastepny pojazd jedzie wolniej od naszego
#                             if next_vehicle[j]>length
#                                 pos_next_vehicle=next_vehicle[j]%length
#                             else
#                                 pos_next_vehicle=next_vehicle[j]
#                             end
#                             if velocities[j, pos_next_vehicle]<velocities[j,i]
#                                 lane_changes[j,i]=1
#                                 if next_vehicle[j+1] > i + velocities[j, i]
#                                     accelerate(velocities, i, j, next_vehicle[j+1], max_velocity)
#                                     slow_down_random(velocities, i, j, p)
#                                     next_vehicle[j+1] = i + ceil(velocities[j,i]/2)
#                                 else
#                                     slow_down_car(velocities, i, j, next_vehicle[j+1])
#                                     slow_down_random(velocities, i, j, p)
#                                     next_vehicle[j+1] = i + ceil(velocities[j,i]/2)    
#                                 end
#                             end
#                         else 
#                             slow_down_car(velocities, i, j, next_vehicle[j])
#                             slow_down_random(velocities, i, j, p)      
#                         end
#                     end
#                 end                
            end
        end
    end
end

function update_positions_open(vehicles, velocities, lane_changes)
        """
        
        """
    length = size(velocities)[2]
    lines = size(velocities)[1]
    
    # If rightmost vehicle goes through periodic boundary, avoid updating it second time by marking new position as "already checked"
    # At the beginning avoid_pos holds zeroes.
    
    # To avoid crashes occuring from passing the periodic boundary, we add to our diagram vehicles from front at the last possible moment,
#     temp_update_vel=fill(0, (lines, 5))
#     temp_update_veh=fill(0, (lines, 5))
#     old_state_veh=copy(vehicles)
#     old_state_vel=copy(velocities)
    for i = length:-1:1
        for j = lines:-1:1
            if (vehicles[j, i] != 0) && (velocities[j, i] != 0)
                if (i+velocities[j, i]) > length
                    ##DELETE CAR FROM EXISTENCE
                    #if vehicles[j+lane_change, new_pos] == 1
                        #println("Car from $j, $i crashed into $(j+lane_change), $(i+velocities[j, i]), $lane_change, $(velocities[j,i])")
                        #println("CRASH REPORT")
                        #print_state(vehicles, velocities, lane_changes)
                    #end
                    ##vehicles[j+lane_change, new_pos] = 1
                    ##velocities[j+lane_change, new_pos] = velocities[j, i]
                    ##avoid_pos[j+lane_change, new_pos]=1
                    vehicles[j, i] = 0
                    velocities[j, i] = 0
                    lane_changes[j, i] = 0
                else
                    lane_change=lane_changes[j, i]
                    if vehicles[j+lane_change, (i+velocities[j, i])] == 1
                        println("Car from $j, $i crashed into $(j+lane_change), $(i+velocities[j, i]), $lane_change, $(velocities[j,i])")
                        println("CRASH REPORT")
                        print_state(vehicles, velocities, lane_changes)
                    end
                    vehicles[j+lane_change, (i+velocities[j, i])] = 1
                    velocities[j+lane_change, (i+velocities[j, i])] = velocities[j, i]
                    vehicles[j, i]=0
                    velocities[j, i]=0
                    lane_changes[j, i] = 0
                end
            end
        end
    end
end

update_positions_open (generic function with 1 method)

In [8]:
function check_right_mirror(vehicles, velocities, next_vehicle, row, pos)
    """
    Function checks if we are able to change lane.
    First - check if next vehicle is further than our "range"
    Then - check every space from our goal, up to (X - now 4) places back.
    
    """
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    if velocities[row, pos]>0
        flag=0
        if next_vehicle[row-1]<pos+velocities[row,pos]
            flag=1
        end
        for p = pos-4:pos+velocities[row,pos]
            if p<1
                tp=p+length
            elseif p>length
                tp=p%length
            else
                tp=p    
            end
            if vehicles[row-1, tp]!=0
                flag=1
            end
        end
        if flag==0
            return "safe"
        else
            return "unsafe"
        end
    end
end

function check_left_mirror(vehicles, velocities, next_vehicle, row, pos)
    """
    Function checks if we are able to change lane.
    First - check if next vehicle is further than our "range"
    Then - check every space from our goal, up to (X - now 4) places back.
    """
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    if velocities[row, pos]>0
        flag=0
        if next_vehicle[row+1]<pos+velocities[row,pos]
            flag=1
        end
        for p = pos-4:pos+velocities[row,pos]+1
            if p<1
                tp=p+length
            elseif p>length
                tp=p%length
            else
                tp=p    
            end
            if vehicles[row+1, tp]!=0
                flag=1
            end
        end
        if flag==0
            return "safe"
        else
            return "unsafe"
        end
    end
end

check_left_mirror (generic function with 1 method)

In [7]:
# try to accelerate
function accelerate(velocities, pos, row, next_vehicle, max_velocity)
    """
    If velocity is smaller than max_velocity chceck if collision occurs.
    If no collision => accelerate
    """
    if velocities[row,pos] < max_velocity
        if pos + velocities[row,pos] + 1 < next_vehicle
            velocities[row,pos] += 1
        end
    end
end


# slow down due to next car
function slow_down_car(velocities, pos, row, next_vehicle)
    """
    Slow down to land on position (next_vehicle - 1) after move.
    """
    velocities[row,pos] = next_vehicle - pos - 1
end


# slow down randomly
function slow_down_random(velocities, pos, row, p)
    """
    Random event of slowing down with probability p. 
    Velocity cannot drop below 0.
    """
    if Random.rand() < p
        if velocities[row,pos] > 0
            velocities[row,pos] -= 1
        end
    end
end

slow_down_random (generic function with 1 method)

In [47]:
##MAIN PROGRAM
length = 500
lines = 5
p = 0.5
politeness_factor=0.08
max_velocity = 5 
min_velocity = 0

0

In [59]:
function print_state(vehicles, velocities, lane_changes)
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    for j in lines:-1:1
        str = ""
        for i = 1:length
            if vehicles[j,i] == 0
                str = str * "."
            else
                str = str * string(velocities[j,i])
            end
        end
        println(str)
    end
    str=""
    for i = 1:length
        str=str*"="
    end
    println(str)
end

function licz_auta(vehicles, aut)
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    s=0
    for j in lines:-1:1 
        for i = 1:length
            if vehicles[j,i]!=0
                s+=1
            end
        end
    end
    if s!=aut
        println("ZŻARŁO")
    end
    println(s)
    return s
end

function print_state_save(vehicles, velocities, lane_changes, io)
    length = size(vehicles)[2]
    lines = size(vehicles)[1]
    for j in lines:-1:1
        str = ""
        for i = 1:length
            if vehicles[j,i] == 0
                str = str * "."
            else
                str = str * string(velocities[j,i])
            end
        end
        #println(str)
        write(io, str, "\n")
    end
    str=""
    for i = 1:length
        str=str*"="
    end
    #println(str)
    write(io, str, "\n")
end

print_state_save (generic function with 1 method)

In [25]:
# closed boundary -  multiline

velocities = fill(0, (lines, length))
vehicles = fill(0, (lines, length))
lane_changes=fill(0, (lines,length))

no_cars=length
no_steps=10000
aut=no_cars
vehicles[1, 1:no_cars] .=1
Random.shuffle!(vehicles)
#print_state(vehicles, velocities, lane_changes)
for step = 1:no_steps
    mcs_closed(vehicles, velocities, lane_changes, max_velocity, min_velocity, p, politeness_factor)
end
aut=licz_auta(vehicles, aut)

BoundsError: BoundsError: attempt to access 3×80 Array{Int64,2} at index [1, 1:400]

In [61]:
# closed boundary -  multiline + saved results

velocities = fill(0, (lines, length))
vehicles = fill(0, (lines, length))
lane_changes=fill(0, (lines,length))

no_cars=length
tim=Dates.format(Dates.now(), "HH.MM.SS")
io = open("Results, open $(tim).txt", "w");
no_steps=25000
aut=no_cars
vehicles[1, 1:no_cars] .=1
Random.shuffle!(vehicles)
for step = 1:no_steps
    mcs_closed_save(vehicles, velocities, lane_changes, max_velocity, min_velocity, p, politeness_factor, io)
end
aut=licz_auta(vehicles, aut)

500


500

In [34]:
# open boundary -  multiline

velocities = fill(0, (lines, length))
vehicles = fill(0, (lines, length))
lane_changes=fill(0, (lines,length))

no_steps=10000
#print_state(vehicles, velocities, lane_changes)
for step = 1:no_steps
    for j in lines:-1:1
        if vehicles[j,1]==0 && velocities[j,1]==0
            vehicles[j,1]=1
            velocities[j,1]=1
        end
    end
    mcs_open(vehicles, velocities, lane_changes, max_velocity, min_velocity, p, politeness_factor)
end

1.............................................................................................................
2.............................................................................................................
2.............................................................................................................
01............................................................................................................
0.3...........................................................................................................
1.2...........................................................................................................
1.1...........................................................................................................
0....4........................................................................................................
02..3.........................................................................................................
0

1.2.................4.........................................................................................
02....3..1.....2................4......5.........................4.........4......................4...........
02...2........3....3............4..........4........4.........3....5.....5..........4...............4.........
01..2....3....................................................................................................
0..3......2......3......5...........4.......4........................4.........4......................5.......
0..2...3.........4....4.............4..........4........5........3......5.....4.........4...............5.....
0.2...3.....3.................................................................................................
0.....4.....2.......3........5..........5.......4........................4.........5.......................4..
0....3....3..........4....5.............5..........5.........5......3........4....5.........5................4
1

01...2......4....................................4............5.......................................4.......
0..2..........5...........4.....5........5.........4......4......4......4.....................5.........4.....
1.1....3.....1..3....3........3...........4...........5......3.....3.............4.........5................4.
1.2....3........5....................................4.............4......................................4...
0....3.............5..........5......4........4........4......5......5.............................5........5.
00.2......3...1....3....4........3............5............3....3.....4.....5........4..........5.............
01..3.....3..........4...................................5.............4......................................
1.......4...............5..........5.....4........5........4.......5......4.............................4.....
00...3.......3.2......4.....5.......4..............4..........3....4......5......5.......4...........4........
0

01.1...2.........3.........4....5.............4................5......5.....4.......4...4.............4......5
1.....1..3....4.........4....3...4......2.....4......4......5........4........4..........3...........5........
01.2..2...2...........3.......................................................................................
1.0.1....2..........3..........4.....5............5.................4......4....5...........4.............4...
02.....1....4.....5.........3...4....3....2.......4......5.......4.......4........4.....2...3.............4...
1.1..2..3...3............3.....................................................3..............................
000..2.....3...........4...........4......4............5................4............5..........5.............
1..2....2.......5......5.......4....3...2...3.........4.......4......4.......4........3...3....4..............
00.2...3...2...4............4..........4......................................................................
0

1....3......3...2.......3...4.....5.........5.........4.........4.....5......4.....4...........4.......5......
0.1...2.....................4..................................................................1...3..........
02...3......4.........3.....3........4..............5....................................4..............4.....
02......3......2..3........4....4......4.........5........4.........5......5.....5.....4...........5........5.
0..2....2.......4...............4.............................................................................
0..2....3........1.......4.....4.........4...............4...................................4..2.....4.....4.
0..2.......4.........3.........4....5......4..........4.......5..........5......5.....5....5............4.....
1....2....2.........3...............5............................................................4............
0....2.....3.................3.....4.........5...............4.............................5......3.......5...
0

...

1.2.....2..3....3...............3...1......3.........4.........2.....5..........2..3.......5........2..2......
0....2...............2.............2....4....................5......................4.........................
00.1.2..2..........................2.....1...2.......................4.......4......5..........4..........4...
02..2.....3...3....4.....3.........0.1........3..........3....4..3........5.......2...3.........4.....1..3....
1......2.............................2......5.....................5.....................5.....................
01..2..2..3............2.............2....2....3..................5......4.......4.......4.........3..........
0..2..3......2...4.....3....3......0..2..........3..........4.......4..........3....3....4..........4..1....4.
02.......3.............................3.........4.....................4.....................4........3.......
1.2...2..2...3...........3.............3....3.....3....................4.............4.......5..........4.....
0

1...3.....4...........1.......4.........................4.....4.........5......5............................4.
1..3...3.........4........3...................................4..........5........4........................2..
0..2.......4...4.....3......5.3...3...2............4.....4....2...2....4........5..........5.....5........4...
02.....3......5...................4...............................5..........5......4.........................
02....3...3....4.....3.2.....3...4..........................4.....5...........5.......4......................3
0....2.............3....4........2...2..2..............5.....4..2...3......4.........4..........4.....4.......
1..2......3........3....3.............5.........................4......4..........4.....4.....................
1..3.....3.........4.....2...........5...........................4.....4...........5......5...................
1......3.....4........3.....4...3..3...2..3.................5.....2....4.......4.........5..........5.....5...
0

01......3.....0.....4...................................4...................4.................................
01..2.........2...2.....3........4.............5............4.............2...4.......4...2.....5........5....
1....2.......4........3......4.......4..0000................4.................5......5...........5.......4....
1.1........2..1.........4...................................5...................5.........3...................
0.2...3.........2..........4.........3...4..........4...........4.................5..................5........
02.....3.........3..3....3.......4......0001....................4...........2......5......0.3.........4......4
01.2...........2..3.........5...........3........................4...................5........................
0...2....2...2......3..........4.............4......................4..................4.....3............5...
0..3......3............3....4........2..000.2...........5...........4.........2.........0.0....4..........5...
1

Excessive output truncated after 524307 bytes.

1.1...1.......3.............................4..............5.........................4.....................4..
0....3.....3....3.............5.......4.......3....4.....1.........5.....4......3.....................4......4
01.....4............4..........3.....3.....3...4....4.....4..3....2...2.....1..3...1.......4......4.......5...
01.2...2.........3..............................4...............5.............................................
1.......4.....4....................5......4......4.....3..1...5.........5....5.....4.....3................5...
0.1........5.......4....4.........3.....3.....3....4....5.......2...3...3....2....1.2..........5......4.......
1.2..2...1..........4...............................4.....4..........4.......4................................
02..........5.....4.....................4.....4......5.....2.......4..............4....5......................
1..1............5......4....5........4.....4.....4.....4.....5....3....2...3...3...2..2.....3.......5.....4...
0

In [60]:
# open boundary -  multiline + save results

velocities = fill(0, (lines, length))
vehicles = fill(0, (lines, length))
lane_changes=fill(0, (lines,length))
tim=Dates.format(Dates.now(), "HH.MM.SS")
io = open("Results, open $(tim).txt", "w");
no_steps=20000
#print_state(vehicles, velocities, lane_changes)
for step = 1:no_steps
    for j in lines:-1:1
        if vehicles[j,1]==0 && velocities[j,1]==0
            vehicles[j,1]=1
            velocities[j,1]=1
        end
    end
    mcs_open_save(vehicles, velocities, lane_changes, max_velocity, min_velocity, p, politeness_factor, io)
end

Car from 4, 496 crashed into 4, 500, 0, 4
CRASH REPORT
1...2...3...2.......2.....4........................................................5...................................5...........5...........................................4...........4.......................................................................4.2..............................................5...............................................................1............................................................................................................4........2.........
01..2....2..............4...............5.........................................5..........................5.........4.............5....................3........................4.............................................4.....5..............2.2.........2..........3.............5......5...............3..............5..................................5......4........5........3.................4................................4...........

In [212]:
velocities = fill(0, (lines, length))
vehicles = fill(0, (lines, length))
lane_changes=fill(0, (lines,length))

no_steps=100
scenario_no=5
load_scenario(vehicles, velocities, lane_changes, scenario_no)
#print_state(vehicles, velocities, lane_changes)
for step = 1:no_steps
    mcs_closed(vehicles, velocities, lane_changes, max_velocity, min_velocity, p)
end

....4.....5.....5.....4.....4.....5......5......................................
....5.....5.....5.....4.....5.....5...........1.0001............................
....4.....4.....5.....5.....5.....5.........00000001............................
........4......5.....4....5.....4......5......4.................................
.........4.....5.....3....5......4.....5.......0000.2...........................
........5.....5......5.....5.....5.....4....0000001.1...........................
............5.......4....5.....3....1.......4.....5.............................
.............5......5...3......4.....3......2..0000...3.........................
.............4.....2......5.....5.....4....0000001.1.2..........................
.................5......5.....3...2..1..3.......5......5........................
..................4........4.......5......4...00001......4......................
.................3...2...5.....5.....5.....000001.1.1..2........................
......................4.....

4....3....5.......4....4....4........2.....5.....4...........4.....4....5.......
.....5.....5......5..........4.......5.........5..........3....4...........5....
..5.....5......4.....5.....4.........5.......4.......4...........5.......5......
....3...4......5......4....4....5......3........4....4....5......5.....5.....5..
4.........5.....5......4.........4........5.........5..............5............
.......5.....4.....5......5....5..........5......5.......4...4........5.......4.
..4....3....4.......5.....4....5.....4....4.........4....5.....5......5.....4...
....5..........4.....5.....3...5.....5.........5.........5...4..........5.......
..4.........4....4......3...........4..........4......4..........5.........5....
5.....4...4.....5........4....4.....4....3....5.........4.....5.....4......3....
.........5.........4......1.........5.....5.........4............5...........4..
5.....4...5.....4....5.....2..4.........4..........5......4...5.......5.........
.....1........5......5......

..5......4.....5.....5.....3..4...........5.................5......4.......5....
.......4.....5......5.....1.......4.......5......5.............2...4.....5......
5.......3....4......5......5.......2....3...3...4.........5......5........5.....
4......5.....4......5.....4.......5............5.................4.....4........
...........4......5......1.1..3.......5........5......5..........3.....4......5.
.....5.....4.....5.......4......5....3.....3...4....4..........5......5........4
....5.......4....5.......4....5........5............4................5.....5....
...5...........5.......4..1.2....3...4.....5........4......5........3......5....
...5......4....5......2......5..........4.....4....4....5...........5......5....
5........5......5.....4......5.....4........5...........5.................5.....
5.......5...........4......1..3.....3....5......4.......5.......5......4........
5.......5.....5.....3...2..5......4.........5.....4....5.....5...........5......
.....5........5......4....5.

In [200]:
function load_scenario(vehicles, velocities, lane_changes, no)
    if no == 1
        vehicles[2, 1]=1
        velocities[2, 1]=5
        vehicles[1, 3]=1
        velocities[1,3]=0
        vehicles[2, 11]=1
        velocities[2, 11]=5
        vehicles[1, 13]=1
        velocities[1,13]=0
    end
    if no==2
        vehicles[1, 1]=1
        velocities[1, 1]=5
        vehicles[1, 3]=1
        velocities[1,3]=2
        vehicles[1, 9]=1
        velocities[1,9]=3
        vehicles[1, 14]=1
        velocities[1,14]=1
        vehicles[2,1]=1
        velocities[2,1]=5
        vehicles[2,9]=1
        velocities[2,9]=2
        vehicles[2,7]=1
        velocities[2,7]=2
        vehicles[2,5]=1
        velocities[2,5]=2
        
    end
    if no==3
        vehicles[2, 1]=1
        velocities[2, 1]=5
        vehicles[1, 4]=1
        velocities[1, 4]=2
        vehicles[1, 6]=1
        velocities[1, 6]=1
        vehicles[1, 76]=1
        velocities[1, 76]=5
        vehicles[2, 76]=1
        velocities[2, 76]=5
    end
    if no==4
        vehicles[2, 80]=1
        velocities[2, 80]=1
        vehicles[1, 80]=1
        velocities[1, 80]=1
        vehicles[1, 2]=1
        velocities[1, 2]=0
        vehicles[2, 2]=1
        velocities[2, 2]=0
        vehicles[1, 3]=1
        velocities[1, 3]=1
        vehicles[2, 3]=1
        velocities[2, 3]=1
    end
    if no==5
        vehicles[2, 52]=1
        velocities[2, 52]=0
        vehicles[2, 51]=1
        velocities[2, 51]=0
        vehicles[2, 50]=1
        velocities[2, 50]=0
        vehicles[2, 49]=1
        velocities[2, 49]=0
        vehicles[1, 52]=1
        velocities[1, 52]=0
        vehicles[1, 51]=1
        velocities[1, 51]=0
        vehicles[1, 50]=1
        velocities[1, 50]=0
        vehicles[1, 49]=1
        velocities[1, 49]=0
        vehicles[1, 48]=1
        velocities[1, 48]=0
        vehicles[1, 47]=1
        velocities[1, 47]=0
        vehicles[1, 46]=1
        velocities[1, 46]=0
        vehicles[1, 45]=1
        velocities[1, 45]=0
        
        vehicles[2, 47]=1
        velocities[2, 47]=0
        vehicles[3, 42]=1
        velocities[3, 42]=5
        

        vehicles[3, 35]=1
        velocities[3, 35]=5
        vehicles[2, 35]=1
        velocities[2, 35]=5
        vehicles[1, 35]=1
        velocities[1, 35]=5
        
        vehicles[3, 29]=1
        velocities[3, 29]=5
        vehicles[2, 29]=1
        velocities[2, 29]=5
        vehicles[1, 29]=1
        velocities[1, 29]=5
        
        vehicles[3, 23]=1
        velocities[3, 23]=5
        vehicles[2, 23]=1
        velocities[2, 23]=5
        vehicles[1, 23]=1
        velocities[1, 23]=5
        
        vehicles[3, 17]=1
        velocities[3, 17]=5
        vehicles[2, 17]=1
        velocities[2, 17]=5
        vehicles[1, 17]=1
        velocities[1, 17]=5
        
        vehicles[3, 11]=1
        velocities[3, 11]=5
        vehicles[2, 11]=1
        velocities[2, 11]=5
        vehicles[1, 11]=1
        velocities[1, 11]=5
        
        vehicles[3, 5]=1
        velocities[3, 5]=5
        vehicles[2, 5]=1
        velocities[2, 5]=5
        vehicles[1, 5]=1
        velocities[1, 5]=5
        
        
    end
end

load_scenario (generic function with 1 method)

In [201]:
for p = 79:83
    if p>80
        tp=p%80
    else
        tp=p
    end
    println(tp)
end

println(ceil(45+1/2)<=47)

79
80
1
2
3
true


In [202]:
velocities = fill(0, (lines, length))
vehicles = fill(0, (lines, length))
lane_changes=fill(0, (lines,length))

lane=fill("", (3,1))
lane[3]=".....1.2.....2..2..1.0...2....2.....2...........5.....1.00.0.2..2..2..1.00.....3"
lane[2]="..1.10.1..3.......1.11.01.2..3.......4...2.....3...3...1..2..01..1.2...21.11.2.."
lane[1]="1.2....1.1.1..3....1...2..1.1.2..3......2..01.2..3...2..000.00.11.1..2....2..1.0"
for i = 1:3
    for j = 1:length
        if lane[i][j]!='.'
            velocities[i,j]=parse(Int,lane[i][j])
            vehicles[i,j]=1
        end
    end
end
no_steps=10
print_state(vehicles, velocities, lane_changes)
for step = 1:no_steps
    mcs_closed(vehicles, velocities, lane_changes, max_velocity, min_velocity, p)
end           


            
        


.....1.2.....2..2..1.0...2....2.....2...........5.....1.00.0.2..2..2..1.00.....3
..1.10.1..3.......1.11.01.2..3.......4...2.....3...3...1..2..01..1.2...21.11.2..
1.2....1.1.1..3....1...2..1.1.2..3......2..01.2..3...2..000.00.11.1..2....2..1.0
.....1.3.....1..1..1.1...3....3.....1...........4.....1.01.1.2..2..1..0.00.....4
..1.00.2..4.......1.01.01.2..3.......4...3.....3...2...1..1..02..1.3...21.00.3..
1.3....1.0.2..4....2...1..1.1.2..4......2..01.2..3...1..000.00.01.2..1....1..1.0
...2..2...2...2..2..0.1.....4....3...1...5..........2..01.1.1..2..1.1.0.01......
1..001...3....3....00.01.2..3...4...........3.....2..2..2..0.1..1.1...2..001....
.2...2..01...3....2..2..1..0.2..2....3....01.2..3...1.1.001.00.1.2..1.1..1.1..01
.....2..3...3...2..01..2........3...1.1.......5.......00.1.1.2...1.0.01.1.2.....
.1.01.1.....4....1.01.1.2..3...4....4..........4....2..2..01..1..1.2....001.2...
2..2...20.2.....3...2..1.0.1...2..3.....3.1.2..3...1.1.001.000..2..1.0.2..1.1.0.
.......2...3...2..01.2...3..